In [89]:
#Load games

import nflgame
games = nflgame.games(2014)

In [141]:
# Extract features
import re
from collections import defaultdict
import random
import numpy as np


play_features = []
success_labels = []
yard_labels = []
success_cnt = 0

random.seed()
for p in nflgame.combine_plays(games).limit(5000000):
    features = defaultdict(float)
    success = 0
    yards = 0
    desc = ''
    if (p.note == None or p.note== 'TD' or p.note=='INT') \
    and (' punt' not in p.desc) \
    and ('END ' != p.desc[:4]) \
    and ('End ' != p.desc[:4]) \
    and ('Two-Minute Warning' not in p.desc) \
    and ('spiked the ball to stop the clock' not in p.desc) \
    and ('kneels to ' not in p.desc) \
    and ('Delay of Game' not in p.desc)\
    and (p.time is not None)\
    and ('Penalty on' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    and ('sacked at' not in p.desc)\
    and ('Punt formation' not in p.desc)\ # Trick play
    and ('Direct snap to' not in p.desc)\ # Trick play
    and ('Aborted' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    and ('Delay of Game' not in p.desc)\
    :
        
        features['team'] = p.team    
        #features['opponent'] = TODO
        timeclock = p.time.clock.split(':')
        features['time'] = float(timeclock[0])*60 + float(timeclock[1])
        features['quarter'] = p.time.qtr            
        features['position'] = 50-p.yardline.offset
        features['down'] = p.down
        features['togo'] = p.yards_togo

        if 'Shotgun' in p.desc:
            features['shotgun'] = 1
        
        sentences = p.desc.split('. ')
        for i in range(len(sentences)):
            if 'reported in as eligible' in sentences[i]:
                continue
                
            if (re.search(r'in at QB$', desc) is not None):
                continue
            
            if ' in at QB' in sentences[i]:
                sentences[i] = re.sub(r"^.+ in at QB", "", sentences[i]).strip()
                
            desc = sentences[i]
            desc = re.sub(r"\(.+?\)", "", desc).strip()

            if ((re.search(r'to \S+$', desc) is not None) or (re.search(r'^\S+$', desc) is not None)) and (i<len(sentences)-1):
                desc = desc + '.' + re.sub(r"\(.+?\)", "", sentences[i+1]).strip()

            if ((i<len(sentences)-1) and (sentences[i+1][:3] == 'to ')):
                desc = desc + '.' + re.sub(r"\(.+?\)", "", sentences[i+1]).strip()

            if (re.search(r'^\S+\.\S+ ', desc) is not None): 
                break
                
                
        if 'incomplete' in desc:
            features['pass'] = 1
            rematch = re.search(r'incomplete \S+ \S+ to ', desc)
            
            if rematch is None:
                # ball just thrown away, no intended target -> ignore
                continue;
                
            match = rematch.group(0).split()            
            features['passlen'] = match[1]
            features['side'] = match[2]
        else:
            if 'no gain' in desc:
                yards = 0
            else:
                if (p.note!='INT') and ('INTERCEPTED' not in desc):         
                    rematch = re.search(r'[-]?[0-9]+ yard\s?', desc) 
                    match = rematch.group(0)         
                    yards = float(match[:match.find(' ')])
            
            if ' pass ' in desc:
                features['pass'] = 1
                match = re.search(r'pass \S+ \S+', desc).group(0).split()
                if match[1] == 'to':
                    continue
                features['passlen'] = match[1]
                features['side'] = match[2]
            else:
                features['pass'] = 0
                if 'up the middle' in desc:
                    features['side'] = 'middle'
                else:               
                    rematch = re.search(r'^\S+ (scrambles )?\S+ \S+', desc) 
                    if rematch is None:
                        print desc
                        print p.desc
                    offset = 0
                    match = rematch.group(0).split()
                    if match[1] == 'scrambles':
                        features['qbrun'] = 1
                        offset = 1
                    
                    features['side'] = match[1+offset] + ' ' + match[2+offset]
                
            if (p.note=='INT') or ('INTERCEPTED' in desc) :
                success = 0
            elif (p.touchdown == True) and (' fumble' not in p.desc):
                success = 1
                success_cnt += 1
            elif (yards >= p.yards_togo): 
                success = 1
                success_cnt += 1
                
        
                        
        play_features.append(features)
        success_labels.append(success)
        yard_labels.append(yards)

    # Debug information
    #if random.randint(0,1000) < 2:
    #    print desc
    #    print p.desc
    #    print features
    #    print 'SUCCESS:',success,'| YARDS:',yards
    #    print "############################################################"
    
success_labels = np.array(success_labels)
print len(play_features)
        

J.Cutler pass short left to M.Bennett pushed ob at CHI 43 for 17 yards
(6:13) (Shotgun) J.Cutler pass short left to M.Bennett pushed ob at CHI 43 for 17 yards (D.Williams). CHI-M.Slauson was injured during the play. His return is Questionable.
defaultdict(<type 'float'>, {'passlen': u'short', 'shotgun': 1, 'team': u'CHI', 'down': 3, 'togo': 4, 'time': 373.0, 'position': 74, 'quarter': 2, 'pass': 1, 'side': u'left'})
SUCCESS: 1 | YARDS: 17.0
############################################################
C.Henne pass short right to M.Lewis to JAC 27 for 7 yards .
(14:59) C.Henne pass short right to M.Lewis to JAC 27 for 7 yards (B.Fletcher).
defaultdict(<type 'float'>, {'passlen': u'short', 'team': u'JAC', 'down': 1, 'togo': 10, 'time': 899.0, 'position': 80, 'quarter': 1, 'pass': 1, 'side': u'right'})
SUCCESS: 0 | YARDS: 7.0
############################################################

(14:19) J.Scobee 49 yard field goal is GOOD, Center-C.Tinker, Holder-B.Anger.
defaultdict(<type 'float'>

In [136]:
# Encode categorical features
from sklearn.feature_extraction import DictVectorizer
import numpy as np

enc = DictVectorizer()
enc.fit(play_features)  
svm_features = enc.transform(play_features)  


In [137]:
svm_features

<29630x77 sparse matrix of type '<type 'numpy.float64'>'
	with 271577 stored elements in Compressed Sparse Row format>

In [138]:
#Train classifier
from sklearn import svm
clf = svm.SVC()
clf.fit(svm_features, success_labels)  


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [139]:
#Predict result for play
features = defaultdict(float)
features['team'] = "GB"
features['time'] = 425
features['quarter'] = 4 
features['position'] = 10
features['down'] = 3
features['togo'] = 10
features['shotgun'] = 0
features['pass'] = 1
features['passlen'] = 'short'
features['side'] = 'left'
features['qbrun'] = 0
svm_feats = enc.transform(features)

clf.predict(svm_feats)

array([0])

In [140]:
#Simple K-Fold cross validation. 10 folds

from sklearn.cross_validation import KFold
from __future__ import division

cv = KFold(len(success_labels), 10)

#iterate through the training and test cross validation segments and
#run the classifier on each one, aggregating the results into a list
success_cnt = 0
total_cnt = 0
iteration = 0
for traincv, testcv in cv:
    iteration = iteration+1
    print "Iteration #"+str(iteration)+"..."
    X_train = svm_features[traincv]
    Y_train = success_labels[traincv]
    X_test = svm_features[testcv]
    Y_test = success_labels[testcv]
    
    Y_pred = clf.fit(X_train, Y_train).predict(X_test)
    
    for i in range(len(Y_pred)):
        total_cnt += 1
        if Y_pred[i] == Y_test[i]:
            success_cnt += 1
        

#print out the mean of the cross-validated results
print "Accuray:",(success_cnt / total_cnt)*100,"%"

Accuray: 71.5322308471 %
